In [ ]:
CREATE OR REPLACE TRIGGER trg_manage_backpaper
AFTER INSERT OR UPDATE ON RESULT
FOR EACH ROW
DECLARE
  v_count NUMBER;
BEGIN
  IF :NEW.MARKS >= 50 THEN
    -- Delete the row from BACKPAPER if it exists
    DELETE FROM BACKPAPER
    WHERE ROLL = :NEW.ROLL AND SCODE = :NEW.SCODE;

  ELSE
    -- Check if the row already exists
    SELECT COUNT(*) INTO v_count
    FROM BACKPAPER
    WHERE ROLL = :NEW.ROLL AND SCODE = :NEW.SCODE;

    -- Insert only if it does not exist
    IF v_count = 0 THEN
      INSERT INTO BACKPAPER (ROLL, SCODE)
      VALUES (:NEW.ROLL, :NEW.SCODE);
    END IF;
  END IF;
END;
/


In [ ]:
CREATE OR REPLACE TRIGGER trg_log_on_delete_request
AFTER DELETE ON REQUEST
FOR EACH ROW
DECLARE
  v_last_entry_no NUMBER;
BEGIN
  -- Get the current maximum ENTRY_NO from SERVICE_LOG
  SELECT NVL(MAX(ENTRY_NO), 0)
  INTO v_last_entry_no
  FROM SERVICE_LOG;

  -- Insert new log entry with incremented ENTRY_NO
  INSERT INTO SERVICE_LOG (ENTRY_NO, REQ_NO, SERVICE_DT)
  VALUES (v_last_entry_no + 1, :OLD.REQ_NO, SYSDATE);
END;
/
